In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.4 MB/s eta 0:00

In [ ]:
#########

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather", 
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

Loading dataset with config: PatentsConfig(name='sample', version=0.0.0, data_dir='sample', data_files={'train': ['https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather']}, description='Patent data from January 2016, for debugging')


Using metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710


Reading metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710
Filtering train dataset by filing start date: 2016-01-01
Filtering train dataset by filing end date: 2016-01-21
Filtering val dataset by filing start date: 2016-01-22
Filtering val dataset by filing end date: 2016-01-31


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset hupd downloaded and prepared to /root/.cache/huggingface/datasets/HUPD___hupd/sample-23bcfec45c886e8c/0.0.0/6920d2def8fd7767046c0470603357f76866e5a09c97e19571896bfdca521142. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# set the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.w

In [ ]:
def preprocess(example):
    if example['decision'] == 'ACCEPTED':
        label = 0
    elif example['decision'] == 'REJECTED':
        label = 1
    else:
        label = None

    if label is not None:
        encoded = tokenizer.encode_plus(example['abstract'],
                                         max_length=512,
                                         padding='max_length',
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
        return {'input_ids': encoded['input_ids'][0],
                'attention_mask': encoded['attention_mask'][0],
                'label': label}

train_dataset = dataset_dict['train'].filter(lambda example: example['decision'] in ['ACCEPTED', 'REJECTED']).map(preprocess)
val_dataset = dataset_dict['validation'].filter(lambda example: example['decision'] in ['ACCEPTED', 'REJECTED']).map(preprocess)

Filter:   0%|          | 0/16153 [00:00<?, ? examples/s]

Map:   0%|          | 0/8719 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9094 [00:00<?, ? examples/s]

Map:   0%|          | 0/4888 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    return {"accuracy": accuracy}

    
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    #eval_steps=500,   # set evaluation steps
    save_strategy='epoch',   # explicitly set save strategy to steps
    save_steps=500,   # set save steps
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
     #                           'attention_mask': torch.stack([f[1] for f in data]),
     #                           'labels': torch.stack([f[2] for f in data])},
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
print(len(train_dataset["input_ids"][1]))

512


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.504186,0.796440
2,0.480400,0.497279,0.792553
3,0.480400,0.575364,0.792349
4,0.352600,0.638901,0.751227
5,0.352600,0.744562,0.760434
6,0.207800,0.805529,0.752864


TrainOutput(global_step=1638, training_loss=0.33116307800069394, metrics={'train_runtime': 2897.5033, 'train_samples_per_second': 18.055, 'train_steps_per_second': 0.565, 'total_flos': 6929899493289984.0, 'train_loss': 0.33116307800069394, 'epoch': 6.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

output_dir = '/content/drive/MyDrive/model-patent-score'
trainer.save_model(output_dir)

NameError: ignored

In [ ]:
ls /content/drive/MyDrive/model-patent-score


config.json        special_tokens_map.json  tokenizer.json     vocab.txt
pytorch_model.bin  tokenizer_config.json    training_args.bin


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_dir = '/content/drive/MyDrive/model-patent-score'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn.functional as F

abstract = "The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network."
#abstract = "A shoe midsole is composed of a base plate (1), a cover (2), a plurality of blades (3), and liquid (4). The blades (3) are formed in such a manner as to rise within a first region (11) of the base plate (1). The blades (3) are each composed of a plurality of flat-shaped blade elements (32, 33) separated each other by slits (31), and are tilted toward the toe side or the heel side. The flat-shaped blade elements (32, 33) are disposed in such a manner as to be divergent toward the toe side or the heel side. The base plate (1) and the cover (2) are joined together, thereby forming a closed space (5), and the liquid (4) is sealed in the closed space."
inputs = tokenizer.encode_plus(abstract, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

# Get the model prediction
outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
scores = torch.softmax(outputs.logits, dim=1)[0]


probs = F.softmax(scores, dim=0)
# Print the scores
accept_prob = probs[1].item()
print(accept_prob)

print(outputs)
print(scores)
print(probs)

0.4032457172870636
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.3373, -0.4909]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([0.6960, 0.3040], grad_fn=<SelectBackward0>)
tensor([0.5968, 0.4032], grad_fn=<SoftmaxBackward0>)
